In [14]:
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install pyspark


ERROR: Could not find a version that satisfies the requirement pyspark (from versions: none)
ERROR: No matching distribution found for pyspark


In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-11-openjdk-amd64/bin"

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from tabulate import tabulate
import numpy as np

In [2]:
import pyspark
sc = pyspark.SparkContext(appName="FraudDetection")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/26 12:40:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("FraudDetection") \
    .getOrCreate()

spark.sparkContext.getConf().getAll()
spark

In [4]:
import numpy as np 
import pandas as pd 
import json
import matplotlib.pyplot as plt
import warnings
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler,StandardScaler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import avg, stddev, count, hour, col, max
from pyspark.ml import Pipeline
from sklearn.manifold import TSNE
import seaborn as sns
from pyspark.sql.functions import desc  # This line is essential for sorting
import json
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import networkx as nx
from pyspark.sql.functions import hour, dayofweek, month, to_timestamp
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import when
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
warnings.filterwarnings("ignore")
data_path='/kaggle/input/transactions-data-big-data/Transactions Data/Transactions Data'


In [5]:
with open("/kaggle/input/transactions-data-big-data/mcc_codes.json", "r") as f:
    mcc_dict = json.load(f)

# Convert to list of dicts
mcc_list = [{"mcc": int(k), "description": v} for k, v in mcc_dict.items()]
mcc_df = spark.createDataFrame(mcc_list)
mcc_df.printSchema()

root
 |-- description: string (nullable = true)
 |-- mcc: long (nullable = true)



In [6]:
users_df = spark.read.csv(f"{data_path}/users.csv", header=True, inferSchema=True)

# Remove '$' and ',' and cast the columns to double type
users_df = users_df.withColumn("per_capita_income", F.regexp_replace("per_capita_income", "[\$,]", "").cast("double"))
users_df = users_df.withColumn("yearly_income", F.regexp_replace("yearly_income", "[\$,]", "").cast("double"))
users_df = users_df.withColumn("total_debt", F.regexp_replace("total_debt", "[\$,]", "").cast("double"))

print(users_df.count())
# Check the count of rows
users_df.show(3)

1219
+---------+-----------+--------------+----------+-----------+------+-------------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|client_id|current_age|retirement_age|birth_year|birth_month|gender|            address|latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|
+---------+-----------+--------------+----------+-----------+------+-------------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|     1645|         62|            65|      1957|         11|Female|    58 Federal Lane|   41.47|   -81.85|          29692.0|      60541.0|  160456.0|         716|               3|
|     1591|         58|            66|      1961|          6|Female|   2200 Third Drive|   48.28|  -122.62|          16537.0|      33717.0|   58236.0|         698|               6|
|     1959|         46|            59|      1973|          4|  Male|8750 Lake Boulevard|  

In [8]:
transactions_df = spark.read.csv(f"{data_path}/transactions.csv", header=True, inferSchema=True)

# Remove '$' and ',' and cast the 'amount' column to double type
transactions_df = transactions_df.withColumn("amount", F.regexp_replace("amount", "[\$,]", "").cast("double"))

# Ensure 'date' is in timestamp format
transactions_df = transactions_df.withColumn("date", F.to_date("date"))

transactions_df = transactions_df.join(mcc_df, on="mcc", how="left")

# Check the count of rows
print(transactions_df.count())
transactions_df.show(3)

8914963
+----+-------+----------+---------+-------+------+-----------------+-----------+-------------+--------+------+--------------------+
| mcc|     id|      date|client_id|card_id|amount|         use_chip|merchant_id|merchant_city|  errors|target|         description|
+----+-------+----------+---------+-------+------+-----------------+-----------+-------------+--------+------+--------------------+
|3780|7475460|2010-01-01|      957|   4532|-147.0|Swipe Transaction|      44795|   Marysville|No Error|    No|Computer Network ...|
|4121|7475341|2010-01-01|     1797|   1127| 43.33|Swipe Transaction|      33326|      Kahului|No Error|    No|Taxicabs and Limo...|
|4121|7475378|2010-01-01|     1575|   2112| 17.14|Swipe Transaction|      29232|       Osprey|No Error|    No|Taxicabs and Limo...|
+----+-------+----------+---------+-------+------+-----------------+-----------+-------------+--------+------+--------------------+
only showing top 3 rows



In [10]:
# Read the cards data
cards_df = spark.read.csv(f"{data_path}/cards.csv", header=True, inferSchema=True)

# Remove '$' and ',' and cast the 'credit_limit' column to double type
cards_df = cards_df.withColumn("credit_limit", F.regexp_replace("credit_limit", "[\$,]", "").cast("double"))

# Check the count of rows
print(cards_df.count())
cards_df.show(3)

4514
+---------+-------+----------+---------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|client_id|card_id|card_brand|card_type|     card_number|expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|
+---------+-------+----------+---------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|      825|   4524|      Visa|    Debit|4344676511950444|12/2022|623|     YES|               2|     24295.0|       09/2002|                 2008|              No|
|      825|   2731|      Visa|    Debit|4956965974959986|12/2020|393|     YES|               2|     21968.0|       04/2014|                 2014|              No|
|      825|   3701|      Visa|    Debit|4582313478255491|02/2024|719|     YES|               2|     46414.0|       07/2003|                 2004|              No|
+---------+------

In [11]:
trans_cards_df = transactions_df.join(cards_df, on=["client_id", "card_id"], how="left")

final_df = trans_cards_df.join(users_df, on="client_id", how="left")
columns_to_drop = ["id","client_id","card_id","merchant_id","card_on_dark_web_idx","longitude","latitude","date", "card_number", "expires", "cvv", "acct_open_date", "address"]
final_df = final_df.drop(*columns_to_drop)
final_df = final_df.withColumn(
    "merchant_city",
    when(F.lower(F.col("merchant_city")) == "online", "online").otherwise("other")
)
final_df.printSchema()
final_df = final_df.na.fill("Unknown")  # for categorical
final_df = final_df.na.fill(0) 
label_indexer = StringIndexer(inputCol="target", outputCol="label")
final_df = label_indexer.fit(final_df).transform(final_df)
##Convert Categorical Columns
categorical_cols = [col for col, dtype in final_df.dtypes if dtype == 'string' and col != 'target']
indexers = [StringIndexer(inputCol=col, outputCol=col+"_idx").fit(final_df) for col in categorical_cols]

for indexer in indexers:
    final_df = indexer.transform(final_df)

# Drop original string columns (except 'target' which is your label)
final_df = final_df.drop(*categorical_cols)

root
 |-- mcc: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_city: string (nullable = false)
 |-- errors: string (nullable = true)
 |-- target: string (nullable = true)
 |-- description: string (nullable = true)
 |-- card_brand: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- has_chip: string (nullable = true)
 |-- num_cards_issued: integer (nullable = true)
 |-- credit_limit: double (nullable = true)
 |-- year_pin_last_changed: integer (nullable = true)
 |-- card_on_dark_web: string (nullable = true)
 |-- current_age: integer (nullable = true)
 |-- retirement_age: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- birth_month: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- per_capita_income: double (nullable = true)
 |-- yearly_income: double (nullable = true)
 |-- total_debt: double (nullable = true)
 |-- credit_score: integer (nullable = true)
 

In [12]:
feature_cols = [col for col in final_df.columns if col not in ["target", "label" ]]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
final_df = assembler.transform(final_df)


In [17]:
(train_data, validationData, test_data) = final_df.randomSplit([0.11, 0.85, 0.002], seed=123)
training_rdd = train_data.rdd
test_rdd = test_data.rdd
print("Train size:", train_data.count())
print("Test size:", test_data.count())

25/04/26 12:53:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Train size: 1018911


Test size: 18529


In [18]:
print("Number of partitions before repartitioning:", training_rdd.getNumPartitions())
num_partitions = 6  # Change this to the desired number of partitions
training_rdd = training_rdd.repartition(num_partitions)
# New number of partitions
print("Number of partitions after repartitioning:", training_rdd.getNumPartitions())

Number of partitions before repartitioning: 6
Number of partitions after repartitioning: 6


In [19]:
import numpy as np
import heapq

def apply_knn(rdd, query_point, k):
    query_np = np.array(query_point.toArray())
    query_norm = np.linalg.norm(query_np)

    def cosine_similarity(np_vector2):
        dot_product = np.dot(query_np, np_vector2)
        mag2 = np.linalg.norm(np_vector2)
        if query_norm == 0 or mag2 == 0:
            return 0
        return dot_product / (query_norm * mag2)

    def map_phase(split):
        neighbors = []
        for row in split:
            label = row.label
            data_point = np.array(row.features.toArray())
            similarity = cosine_similarity(data_point)
            if len(neighbors) < k:
                heapq.heappush(neighbors, (similarity, label))
            else:
                heapq.heappushpop(neighbors, (similarity, label))
        return [neighbors]

    def reduce_phase(neighbors1, neighbors2):
        merged = neighbors1 + neighbors2
        top_k = heapq.nlargest(k, merged, key=lambda x: x[0])
        return top_k

    def classify_input(neighbors):
        labels = [label for _, label in neighbors]
        labels = np.array(labels).astype(int)
        class_counts = np.bincount(labels)
        most_common_class = np.argmax(class_counts)
        return most_common_class

    mapped_neighbors = rdd.mapPartitions(map_phase)
    final_neighbors = mapped_neighbors.reduce(reduce_phase)
    return classify_input(final_neighbors)


In [24]:
import numpy as np
from tabulate import tabulate
from tqdm import tqdm

def calculate_confusion_matrix(true_labels, predicted_labels, labels):
    num_classes = len(labels)
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)
    label_to_index = {label: i for i, label in enumerate(labels)}

    for true_label, predicted_label in zip(true_labels, predicted_labels):
        true_index = label_to_index[true_label]
        predicted_index = label_to_index[predicted_label]
        confusion_matrix[true_index, predicted_index] += 1
    return confusion_matrix

def calculate_accuracy(confusion_matrix):
    tp_tn = np.trace(confusion_matrix)  # sum of diagonal (true positives + true negatives)
    tp_tn_fp_fn = np.sum(confusion_matrix)  # total elements
    accuracy = tp_tn / tp_tn_fp_fn if tp_tn_fp_fn != 0 else 0
    return accuracy

def calculate_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

def calculate_precision(confusion_matrix, class_index):
    true_positive = confusion_matrix[class_index, class_index]
    column_sum = np.sum(confusion_matrix[:, class_index])
    precision = true_positive / column_sum if column_sum != 0 else 0
    return precision

def calculate_recall(confusion_matrix, class_index):
    true_positive = confusion_matrix[class_index, class_index]
    row_sum = np.sum(confusion_matrix[class_index, :])
    recall = true_positive / row_sum if row_sum != 0 else 0
    return recall

def calculate_macro_average(confusion_matrix, calculate_metric):
    num_classes = len(confusion_matrix)
    metrics = [calculate_metric(confusion_matrix, i) for i in range(num_classes)]
    return np.mean(metrics)

def calculate_micro_average(confusion_matrix, calculate_metric):
    num_classes = len(confusion_matrix)
    true_positives = np.trace(confusion_matrix)
    all_positives = np.sum(confusion_matrix)
    return true_positives / all_positives if all_positives != 0 else 0

def display_confusion_matrix(confusion_matrix, labels):
    print("Confusion Matrix:")
    table = [[''] + labels] + [[labels[i]] + list(confusion_matrix[i]) for i in range(len(labels))]
    print(tabulate(table, tablefmt='grid'))

def evaluate_knn(test_rdd):
    predicted_list = []
    true_list = []
    
    for row in tqdm(test_rdd):
        true_label = row.label
        point = row.features
        knn_predict = apply_knn(training_rdd, point, 3)
        predicted_list.append(knn_predict)
        true_list.append(true_label)
    
    labels = [1, 0]
    confusion_matrix = calculate_confusion_matrix(true_list, predicted_list, labels)

    # Display Confusion Matrix
    display_confusion_matrix(confusion_matrix, labels)

    # Calculate Evaluation Metrics
    accuracy = calculate_accuracy(confusion_matrix)
    macro_precision = calculate_macro_average(confusion_matrix, calculate_precision)
    micro_precision = calculate_micro_average(confusion_matrix, calculate_precision)
    macro_recall = calculate_macro_average(confusion_matrix, calculate_recall)
    micro_recall = calculate_micro_average(confusion_matrix, calculate_recall)

    f1_macro = calculate_f1_score(macro_precision, macro_recall)
    f1_micro = calculate_f1_score(micro_precision, micro_recall)

    # Display Metrics
    metrics_table = [
        ['Accuracy', accuracy],
        ['Macro Precision', macro_precision],
        ['Micro Precision', micro_precision],
        ['Macro Recall', macro_recall],
        ['Micro Recall', micro_recall],
        ['Macro F1', f1_macro],
        ['Micro F1', f1_micro]
    ]
    print(tabulate(metrics_table, tablefmt='grid'))


In [25]:
evaluate_knn(test_rdd.take(3000))

100%|██████████| 3000/3000 [5:36:11<00:00,  6.72s/it]                                               

Confusion Matrix:
+---+---+------+
|   | 1 |    0 |
+---+---+------+
| 1 | 0 |    8 |
+---+---+------+
| 0 | 0 | 2992 |
+---+---+------+
+-----------------+----------+
| Accuracy        | 0.997333 |
+-----------------+----------+
| Macro Precision | 0.498667 |
+-----------------+----------+
| Micro Precision | 0.997333 |
+-----------------+----------+
| Macro Recall    | 0.5      |
+-----------------+----------+
| Micro Recall    | 0.997333 |
+-----------------+----------+
| Macro F1        | 0.499332 |
+-----------------+----------+
| Micro F1        | 0.997333 |
+-----------------+----------+
